In [ ]:
#import the required functions
import numpy as np 
import pandas as pd 
import cv2
import time
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import hog
import os
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
folderpath = "/content/gdrive/MyDrive/PROJECT_D/PINS"
#images_1 = '/content/gdrive/MyDrive/PROJECT_D/PINS/pins_Aaron Paul'
width = 224
height = 224

In [ ]:
#Extracting the names and the images into 2 arrays. note this process takes 32 min
labels = []
data = []
for dirname,_, filenames in tqdm(os.walk(folderpath)):
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename))
        image = cv2.resize(image , (width,height))
        labels.append(dirname.split("/")[-1])
        data.append(image)

In [ ]:
#VGG FACE MODEL
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

# Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))	
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Load VGG Face model weights
model.load_weights('/content/gdrive/MyDrive/PROJECT_D/Part 3 - vgg_face_weights.h5')

In [ ]:
#PLAYING AROUND WITH DATA
print(labels[0:100])

In [ ]:
print(data[0].shape)

In [ ]:
#IMPORT THE VGG MODEL
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [ ]:
#GETTING THE ENCODINGS
img = np.expand_dims(data[0], axis=0)
imgs_representation = vgg_face_descriptor.predict(img)

In [ ]:
print(imgs_representation.shape)

In [ ]:
len(data)

In [ ]:
#APPENDING ALL THE ENCODINGS INTO A ARRAY CALLED ENCODINGS_ARR
encodings_arr = []
for i in range(0,len(data)):
  img = np.expand_dims(data[i], axis=0)
  imgs_representation = vgg_face_descriptor.predict(img)
  encodings_arr.append(imgs_representation)

In [ ]:
print(len(encodings_arr))
print(len(labels))

In [ ]:
#LABEL ENCODER FOR NAMES
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Labels= le.fit_transform(labels)

In [ ]:
#TRAINING AND TESTING
X_train = encodings_arr
Y_train = Labels
X_test = encodings_arr[9000:]
Y_test = Labels[9000:]

In [ ]:
print(X_train[0])
for i in range(0,len(X_train)):
  a = np.squeeze(X_train[i], axis=0)
  X_train[i] = a

print(X_train[0])

for i in range(0,len(X_test)):
  a = np.squeeze(X_test[i], axis=0)
  X_test[i] = a

print(X_test[0])

In [ ]:
#INTRODUCING PCA AND FITTING THE MODEL INTO X_TRAIN
from sklearn.decomposition import PCA
t= time.time()
pca = PCA(n_components=150 , svd_solver='randomized' , whiten=True).fit(X_train)
#print("Time evolved", time.time()-t)

In [ ]:
#APPLYING THE PRE-PROCESSED DATA INTO SVM
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
svm = SVC(kernel='rbf' , class_weight='balanced' , C=1000 , gamma=0.0082)
svm.fit(X_train_pca , Y_train)

In [ ]:
#TRAINING SCORE
print(svm.score(X_train_pca , Y_train))

In [ ]:
#ARTIFICALLY CREATING A HUGE DATASET FOR TRAINING PURPORE SINCE THE IMAGES OF SAME PERSON ARE CONTINUOUS AND WE NEED DIVERSE I/P
X_t = []
Y_t = []
import random
for i in range(0,100000):
  a = random.randint(0,10000)
  for i in range(0,1):
    ind = a + i
    X_t.append(X_train_pca[ind])
    Y_t.append(Y_train[ind])

print(len(X_t))
print(len(Y_t))


In [ ]:
Y_t[0:100]

In [ ]:
svm.fit(X_t , Y_t)

In [ ]:
#SCORE ON TESTING MODEL
print(svm.score(X_test_pca , Y_test))

In [ ]:
#PREDICTION FOR A IMAGE
a = X_test_pca[0]
k = np.expand_dims(a,axis=0)
p = svm.predict(k)

In [ ]:
Y_test[0]
p[0]

In [ ]:
#SEEING THE NUMBER OF CORRECTLY AND FALSELY LABELED ANSWERS
true_ = 0
false_ = 0

for i in range(0,len(X_test_pca)):
  a = X_test_pca[i]
  k = np.expand_dims(a,axis=0)
  p = svm.predict(k)
  if p[0] == Y_test[i]:
    true_ +=1

  else:
    false_ +=1

print(true_)
print(false_)

In [ ]:
Labels[99]

In [ ]:
#CREATING DICTIONARY FOR DIFFERENT labels->Labels and Labels->labels
res_names = dict(zip(Labels, labels))
res_labels = dict(zip(labels, Labels))

In [ ]:
res_names[67]

In [ ]:
print(width)

In [ ]:
#CHECKING ON A IMAGE FROM INTERNET
im = cv2.imread('/content/800px-Mark_Zuckerberg_F8_2019_Keynote_(32830578717)_(cropped).jpg')
image = cv2.resize(im, (224,224), interpolation = cv2.INTER_AREA)
image = np.expand_dims(image, axis=0)
imgs_representation = vgg_face_descriptor.predict(image)
imgs  = pca.transform(imgs_representation)
k = np.expand_dims(imgs,axis=0)
p = svm.predict(imgs)

index = p[0]

#X_train = encodings_arr
print(res_names[index])

In [ ]:
plt.imshow(image[0])

In [ ]:
i = 62

In [ ]:
import sklearn
a = encodings_arr[0]
a = np.expand_dims(a, axis=0)
b = encodings_arr[i]
b = np.expand_dims(b, axis=0)
n = sklearn.metrics.pairwise.cosine_similarity(a,b)
print(n[0][0])

In [ ]:
labels[i]

In [ ]:
#calculate cosine similaarity for images
def cosine_sim(i,j):
  a = encodings_arr[i]
  a = np.expand_dims(a, axis=0)
  b = encodings_arr[j]
  b = np.expand_dims(b, axis=0)
  n = sklearn.metrics.pairwise.cosine_similarity(a,b)
  print(n[0][0])
  if (n[0][0] >= 0.65):
    print("Similar Faces")
    print(labels[i])
  else:
    print("Dissimilar Faces")

In [ ]:
#playing around
a = random.randint(0,100)
b = random.randint(0,100)
cosine_sim(a,b)
print(a,b)